In [1]:
#!pip3 install stocksymbol
#!pip3 install pubproxpy
#!pip3 install yfinance
#!pip3 install termcolor

In [2]:
import yfinance as yf
from stocksymbol import StockSymbol
from pubproxpy import Level, Protocol, ProxyFetcher
import requests
import time
import json
import logging
import socket
import threading
import random
import time
from tqdm import tqdm
from termcolor import colored

In [3]:
#Init
stockData = None
proxy_refresh_rate = 50
thread_size = 120
symbols_api_key = '****'
proxy_api_key = '****'
#https://medium.datadriveninvestor.com/download-list-of-all-stock-symbols-using-this-python-package-12937073b25

In [4]:
#get symboles
ss = StockSymbol(symbols_api_key)
symbols_dict = ss.get_symbol_list(market="US")
symbols = [symbol['symbol'] for symbol in symbols_dict]

In [5]:
def getNewProxy():
    while True:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36',
        }
        params = {
            'api': proxy_api_key,
            'level': 'elite',
            # 'spped': 60,
        }
        result = requests.get('http://pubproxy.com/api/proxy', params=params,  headers=headers).json()
        if "data" in result:
            proxyString = result['data'][0]['type'] + '://' + result['data'][0]['ipPort']
            proxies = {'http': proxyString}
            return proxies
        else:
            time.sleep(2)
            continue


In [6]:
# Proxy Test
# proxies = getNewProxy()
# print(proxies)

In [7]:
def getTicker(symbol, period, proxy):
    stock = yf.Ticker(symbol)
    return stock.history(period=period, proxy=proxy)

In [42]:
def batchTickers(symbols, period):
    results = None
    count = 0
    total = len(symbols)
    proxy = None
    #for symbol in tqdm(symbols): #avoid too many outputs with multithreading
    for symbol in symbols:
        if count % proxy_refresh_rate == 0:
            try:
                proxies = getNewProxy()
                proxy = proxies['http']
            except:
                print('Use Previous Proxy')
                pass
        try:
            ticker = getTicker(symbol, period, proxy).reset_index()
            ticker['symbol'] = symbol
            try:
                if not ticker.empty:
                    results = results.append(ticker, ignore_index=True)
                    
            except:
                #print('Start Over')
                results = ticker
            #time.sleep(random.randrange(4))
        except:
            print('skipped one symbol: {}'.format(symbol))
        count = count + 1
    return results

In [44]:
def threadedBatchTickers(symbols, period, df_list = [], i=1):
    if(len(symbols) <= thread_size):
        data = batchTickers(symbols, period)
        df_list.append(data)
        print(colored('Completed for job id {}'.format(i), 'red'))
        print(colored('Number of Stocks: {}'.format(len(data['symbol'].unique()), 'red')))
    else:
        split_point = int(len(symbols)/2)
        sym1 = symbols[:split_point]
        sym2 = symbols[split_point:]
        df_list1 = []
        df_list2 = []
        t1 = threading.Thread(target=threadedBatchTickers, name='t{}'.format(i*2), args=(sym1, period, df_list1, i*2))
        t2 = threading.Thread(target=threadedBatchTickers, name='t{}'.format(i*2+1), args=(sym2, period, df_list2, i*2+1))
        t1.start()
        t2.start()
        t1.join()
        t2.join()
        df_list.extend(df_list1)
        df_list.extend(df_list2)

In [45]:
stockDataList = []
threadedBatchTickers(symbols, '6mo', df_list = stockDataList)

PSTH: No data found, symbol may be delisted
ENDP: No data found, symbol may be delisted
MITC: No data found, symbol may be delisted
CND: No data found, symbol may be delisted
XPOA.U: No data found, symbol may be delisted
SYTE: No data found, symbol may be delisted
CLAQU: No data found, symbol may be delisted
HMHC: No data found, symbol may be delisted
SWET: No data found, symbol may be delisted
CND.U: No data found, symbol may be delisted
SLBG: No data found, symbol may be delisted
HTA: No data found, symbol may be delisted
TACO: No data found for this date range, symbol may be delisted
CLIM.U: No data found, symbol may be delistedXPOA: No data found, symbol may be delisted

CLAQ: No data found, symbol may be delisted
SWETU: No data found, symbol may be delisted
RMO: No data found, symbol may be delisted
PORT.U: No data found, symbol may be delisted
FVE: No data found, symbol may be delisted
ROLL: No data found, symbol may be delisted
DNAA: No data found, symbol may be delisted
ENJY: N

MWXRF: No data found, symbol may be delisted
BKEP: No data found, symbol may be delisted
MNR: No data found, symbol may be delisted
USAK: No data found, symbol may be delistedREVHU: No data found, symbol may be delisted

TBK: No data found, symbol may be delisted
UEPS: No data found, symbol may be delisted
CLRM: No data found, symbol may be delisted
BLSA: No data found, symbol may be delisted
ZSAN: No data found, symbol may be delisted
PAQCU: No data found, symbol may be delisted
ESSC: No data found, symbol may be delistedDRE: No data found, symbol may be delisted
SC: No data found, symbol may be delisted

TRIT: Period '6mo' is invalid, must be one of ['1d', '5d']
ARGUU: No data found, symbol may be delisted
HRVOF: No data found, symbol may be delisted
REVH: No data found, symbol may be delisted
LHAA: No data found, symbol may be delisted
MBAC.U: No data found, symbol may be delisted
CLRMU: No data found, symbol may be delisted
MCFE: No data found, symbol may be delisted
STXB: No data 

KOR: No data found for this date range, symbol may be delisted
ETTX: No data found, symbol may be delisted
YSAC.U: No data found, symbol may be delisted
ACKIU: No data found, symbol may be delisted
ZWRK: No data found, symbol may be delisted
NES: No data found, symbol may be delisted
AFAQ: No data found, symbol may be delisted
RCFA.U: No data found, symbol may be delisted
VOLT: No data found, symbol may be delisted
CLCN: No data found, symbol may be delistedMOG.A: No data found, symbol may be delisted

BOAS.U: No data found, symbol may be delisted
BIF: No data found, symbol may be delisted
HCIC: No data found, symbol may be delisted
RRD: No data found, symbol may be delisted
SHI: No data found, symbol may be delisted
SBKK: No data found, symbol may be delisted
AGCB: No data found, symbol may be delisted
JOSMF: No data found, symbol may be delistedALRT: No data found, symbol may be delisted

BOAS: No data found, symbol may be delisted
CNSNF: No data found, symbol may be delisted
CFV: No

RJAC.U: No data found, symbol may be delisted
GRFXY: No data found, symbol may be delisted
GACQU: No data found, symbol may be delisted
PZN: No data found, symbol may be delisted
VOXCF: No data found, symbol may be delisted
ITHXU: No data found, symbol may be delisted
EQD: No data found, symbol may be delisted
SBII.U: No data found, symbol may be delisted
SNII: No data found, symbol may be delistedQFTA.U: No data found, symbol may be delisted

SIMP: No data found, symbol may be delisted
VMACU: No data found, symbol may be delisted
GGPIU: No data found, symbol may be delisted
OPA.U: No data found, symbol may be delisted
SBII: No data found, symbol may be delisted
PPGHU: No data found, symbol may be delisted
SNII.U: No data found, symbol may be delistedIGNY: No data found, symbol may be delisted
EQD.U: No data found, symbol may be delisted

BF.A: No data found, symbol may be delisted
VMAC: No data found, symbol may be delisted
SZC: No data found, symbol may be delisted
GGPI: No data foun

NXGWF: No data found, symbol may be delisted
FTCV: No data found, symbol may be delisted
OTRA: No data found, symbol may be delistedWSFT: No data found, symbol may be delisted
DCRN: No data found, symbol may be delisted

NSTB.U: No data found, symbol may be delisted
FLME.U: No data found, symbol may be delisted
RACB: No data found, symbol may be delisted
JOFFU: No data found, symbol may be delisted
SRSA: No data found, symbol may be delistedCTEK: No data found, symbol may be delisted

NEW: No data found, symbol may be delisted
FTCVU: No data found, symbol may be delisted
MN: No data found, symbol may be delisted
DWIN.U: No data found, symbol may be delistedLOKM.U: No data found, symbol may be delisted

CWEN.A: No data found, symbol may be delisted
DWIN: No data found, symbol may be delisted
SGMS: No data found, symbol may be delisted
EQHA: No data found, symbol may be delisted
PEYE: No data found, symbol may be delisted
KWEMF: No data found, symbol may be delisted
PIPP: No data found, 

FVIV.U: No data found, symbol may be delisted
CLSN: No data found, symbol may be delisted
BENEU: No data found, symbol may be delisted
TMPMU: No data found, symbol may be delisted
HIII: No data found, symbol may be delisted
HNGR: No data found, symbol may be delisted
PING: No data found, symbol may be delisted
HIIIU: No data found, symbol may be delisted
JUN.U: No data found, symbol may be delisted
NRZ: No data found, symbol may be delisted
CDEV: No data found, symbol may be delisted
RBAC.U: No data found, symbol may be delisted
ATSPT: No data found, symbol may be delisted
RLGMF: No data found, symbol may be delisted
LHC.U: No data found, symbol may be delisted
IIN: No data found, symbol may be delisted
NTUS: No data found, symbol may be delisted
Completed for job id 195
Number of Stocks: 47
RBAC: No data found, symbol may be delisted
ATSPU: No data found, symbol may be delisted
SRRA: No data found, symbol may be delisted
Completed for job id 208
Number of Stocks: 49
YMTX: No data foun

WTRH: No data found, symbol may be delistedHWKMF: No data found, symbol may be delisted
Completed for job id 214

Number of Stocks: 62
TGA: No data found, symbol may be delisted
Completed for job id 206
Number of Stocks: 58
AFIN: No data found, symbol may be delisted
Completed for job id 248
FSNB.U: No data found, symbol may be delisted
Number of Stocks: 65
Completed for job id 178
Number of Stocks: 59
Completed for job id 158
Completed for job id 181
Number of Stocks: 63
Number of Stocks: 60
CONE: No data found, symbol may be delisted
Completed for job id 253
Number of Stocks: 66
GPL: No data found, symbol may be delisted
Completed for job id 223
Number of Stocks: 62
IPOF: No data found, symbol may be delisted
Completed for job id 182
Number of Stocks: 57
Completed for job id 226
Number of Stocks: 60
OAS: No data found, symbol may be delisted
Completed for job id 157
Number of Stocks: 62
NTTHF: No data found, symbol may be delisted
Completed for job id 177
Number of Stocks: 56
Complet

In [50]:
# print(stockDataList.memory_usage(index=True).sum() / 1000000)
print(len(stockDataList))
stockData = None
for df in stockDataList:
    try:
        stockData = stockData.append(df)
    except:
        stockData = df
    

128


In [54]:
#stockData = batchTickers(symbols, '6mo')
print('MB of Memory Taken: ', stockData.memory_usage(index=True).sum() / 1000000)
print('Total num of stocks:', len(stockData['symbol'].unique()))

MB of Memory Taken:  84.436088
Total num of stocks: 7720


In [ ]:
#Notable Warnings:
#TRIT: Period '6mo' is invalid, must be one of ['1d', '5d']
#GSKY: No data found for this date range, symbol may be delisted
#VWTR: No data found for this date range, symbol may be delisted
#BF.B: No data found for this date range, symbol may be delisted
#MYJ: No data found for this date range, symbol may be delisted
#GEF.B: No data found for this date range, symbol may be delisted

In [28]:
def listadd(a,b,c):
    c.extend(a)
    c.extend(b)

In [29]:
a = [1]
b=[2]
c = []
listadd(a,b,c)
print(c)

[1, 2]
